<a href="https://colab.research.google.com/github/VVasanth/ReinforcementLearning/blob/main/TicTacToeAgent_Test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "gym>=0.21.0"
!pip install tf-agents

!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 626 kB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.23.1-py3-none-any.whl size=701377 sha256=9f4875665a0ba3c129520471ddca4bef1cd00f126dc2799b1869e1152952e2d7
  Stored in directory: /root/.cache/pip/wheels/e3/33/04/6723848e46f0f1ebe794bb329b7c761c3329a0d7ffade99da7
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.0 MB/s 
     |████████████████████████████████| 18.3 MB 91 kB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts
import numpy as np
import random
from itertools import product

In [ ]:
import tempfile
import os
basedir = '/content/drive/My Drive/Colab Notebooks/ReinforcementLearning/TicTacToe'

In [ ]:
policy_dir = os.path.join(basedir, 'policy')

In [ ]:
import tensorflow as tf
saved_policy = tf.saved_model.load(policy_dir)

In [ ]:
def generatePositions(size):
        # initialize N
        # All possible N combination tuples
        # Using list comprehension + product()
        res = [ele for ele in product(range(0, size), repeat=size-1)]
        return res

def showBoard(board):
    # p1: x  p2: o
    for i in range(0, 3):
        print('-------------')
        out = '| '
        for j in range(0, 3):
            if board[i, j] == 1:
                token = 'x'
            if board[i, j] == -1:
                token = 'o'
            if board[i, j] == 0:
                token = ' '
            out += token + ' | '
        print(out)
    print('-------------')

In [ ]:
def genTimeStep(state):
  state = np.array([state])
  state_reshaped = state.reshape(-1,9)
  tensor = tf.convert_to_tensor(state_reshaped)
  tstep = ts.transition(tensor, reward=tf.constant([1.0]), discount=tf.constant([1.0]))
  return tstep

def updateAction(board, position, symbol):
    board[position] = symbol
    return board

def checkGameStatus(board):
        youWinMsg = "You Win!!!!"
        youLoseMsg = "You Lose :-("
        itsTieMsg = "Its a Tie !!!"
        # row
        for i in range(board.shape[0]):
            if sum(board[i, :]) == 3:                
                return youLoseMsg
            if sum(board[i, :]) == -3:                
                return youWinMsg
        # col
        for i in range(board.shape[1]):
            if sum(board[:, i]) == 3:                
                return youLoseMsg
            if sum(board[:, i]) == -3:              
                return youWinMsg
        # diagonal
        diag_sum1 = sum([board[i, i] for i in range(board.shape[1])])
        diag_sum2 = sum([board[i, board.shape[1] - i - 1] for i in range(board.shape[1])])
        diag_sum = max(diag_sum1, diag_sum2)
        if diag_sum == 3:            
            return youLoseMsg
        if diag_sum == -3:
            return youWinMsg

        # tie
        # no available positions
        if len(get_availablePositions(board)) == 0:
            return itsTieMsg
        # not end
        return 0

def get_availablePositions(board):
        positions = []
        for i in range(board.shape[0]):
            for j in range(board.shape[1]):
                if board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions

In [ ]:
import numpy as np
board = np.zeros((3, 3))
base_positions = generatePositions(3)
states = np.zeros(9)
showBoard(board)
contFlag=True

-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------


In [ ]:
while contFlag:
    row = int(input("Input your action row:"))
    col = int(input("Input your action col:"))
    user_position = (row, col)
    board = updateAction(board, user_position, -1)
    pos_index = base_positions.index(user_position)
    states[pos_index] = 1
    gameStatus = checkGameStatus(board)
    if gameStatus !=0:
      print(gameStatus)
      contFlag=0
    else:
      upd_state = board.flatten()
      ts_state = genTimeStep(upd_state)
      action_step = saved_policy.action(ts_state)
      ag_action = action_step.action
      print(ag_action)
      ag_position = base_positions[ag_action]
      board = updateAction(board, ag_position, 1)
      showBoard(board)

Input your action row:0
Input your action col:0
tf.Tensor([3], shape=(1,), dtype=int32)


TypeError: ignored